# 파인튜닝 튜토리얼

<img src="https://i.imgur.com/sLyyk7l.png" width="700px" />

라이브러리 설치 후 런타임 재시작

In [3]:
!pip install transformers datasets accelerate -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 930.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 912.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 856.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 791.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 1.1 MB/s eta 0:00:00


## 1.작업정의

사전 학습된 언어생성 모델을 가져와서 질의 응답 데이터셋을 가지고 파인튜닝하기

## 2.학습 데이터 로딩

In [4]:
from datasets import load_dataset

train_dataset = load_dataset('Aeala/ShareGPT_Vicuna_unfiltered', split="train[:1000]")
train_dataset


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['conversations', 'id'],
    num_rows: 1000
})

In [5]:
train_dataset[0]["conversations"]

[{'from': 'human',
  'markdown': None,
  'text': None,
  'value': "Summarize the main ideas of Jeff Walker's Product Launch Formula into bullet points as it pertains to a growth marketing agency implementing these strategies and tactics for their clients..."},
 {'from': 'gpt',
  'markdown': None,
  'text': None,
  'value': "Here are the main ideas of Jeff Walker's Product Launch Formula that can be applied by a growth marketing agency for their clients:\n\n1. Identify the target audience and their needs: Understand the ideal customer for the product or service, and create a messaging that resonates with them.\n2. Pre-launch: Build anticipation and excitement for the launch by creating buzz, gathering testimonials and case studies, and using social media to create awareness.\n3. Launch: Use a well-crafted launch sequence to maximize sales and conversions. This can include offering bonuses, creating scarcity, and using a deadline to create urgency.\n4. Post-launch: Follow up with custome

In [6]:
len(train_dataset[0]["conversations"])

12

## 3.토크나이저 로딩

### 기본 용법

In [7]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
tokenizer

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

```
'GPT2TokenizerFast' 토크나이저는 Rust로 작성된 토크나이저를 Python에서 사용할 수 있게 만든 것으로, 일반 토크나이저보다 속도가 빠르다.
```

## 4.모델 로딩

In [8]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained('gpt2')
model


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

## 5.학습 데이터 전처리

### 5.1. 토크나이징

#### 기본용법

```python
[{'from': 'human',
  'markdown': None,
  'text': None,
  'value': 'Sure, I can do that. What new technology would you like me to review?'},
 {'from': 'gpt',
  'markdown': None,
  'text': None,
  'value': 'Samsung S22 Ultra...'},
```

In [13]:
"""
HUMAN: ~~~~  \n
CHATBOT: ~~~~ \n
"""
tokenizer.pad_token = tokenizer.eos_token

# train_dataset_one = train_dataset.select(range(10))

def tokenizer_function(examples):
    conversations = examples["conversations"]

    text = ''

    for item in conversations:
        print(item)
        if item['from'] == 'human':
            text += f"HUMAN: {item['value']} \n"
        else:
            text += f"CHATBOT: {item['value']} \n"


    return tokenizer(text, padding='max_length', truncation=True, max_length=512)

tokenized_datasets = train_dataset.map(
    tokenizer_function,
    remove_columns = ['conversations', 'id']
)


Output hidden; open in https://colab.research.google.com to view.

In [14]:
tokenized_datasets


Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1000
})

### 5.2. 배치 처리 (DataCollator)

DataCollatorForLanguageModeling

In [15]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
data_collator

DataCollatorForLanguageModeling(tokenizer=GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}, mlm=False, mlm_probability=0.15, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt')

## 6.모델 학습

In [16]:
from transformers import Trainer, TrainingArguments
# save_total_limit: 모델 저장할 때, 최신 모델을 몇개 저장할 것인가

training_args = TrainingArguments(
    output_dir='./gpt_vicuna_finetuned',
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=8,
    save_steps=1000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    train_dataset= tokenized_datasets,
    args=training_args
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=250, training_loss=2.0068328857421873, metrics={'train_runtime': 90.4922, 'train_samples_per_second': 22.101, 'train_steps_per_second': 2.763, 'total_flos': 522584064000000.0, 'train_loss': 2.0068328857421873, 'epoch': 2.0})

In [22]:
# 1) 질의 문장
# input_text = "What is the deep learning?"
# input_text = "please, explain language model process"
input_text = "Show me the python code example."

# 2) 토큰화하고 텐서로 변환
input_ids = tokenizer.encode(input_text, return_tensors='pt')
input_ids = input_ids.to("cuda")

# 3) 생성 옵션을 설정하고, 텍스트 생성
max_length = 500
model = model.to('cuda')

sample_outputs = model.generate(
    input_ids,
    do_sample=True,
    max_length=max_length,
    temperature=0.7
)

# 4) 생성된 텍스트 디코딩
for i, sample_output in enumerate(sample_outputs):
    print('Generated text: ')
    print(tokenizer.decode(sample_output, skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text: 
Show me the python code example.
```
from krlearn import model
from kr_learn_learn_model
from kr_learn_model.learn_model import kr_model.classifier

# Initialize the model and add the model
model = model.initialize()

# Create the data
data = kr_learn_model.models.Data('kr_tensor_data')

# Create the model and add the model
model = kr_learn_model.models.Data('kr_tensor_model')

model_tensor = kr_tensor.features.data.train.apply(kr_tensor, tensor)

# Define the training function
def training_function(self):
       kr_model = kr_learn_model.classes.Kernel.UniformClassifier

kr_tensor = kr_tensor.model.data.train.apply(kr_tensor, tensor)

# Define the model
model = kr_learn_model.models.Data.train.apply(kr_tensor, kr_tensor_data)
model_t = model_tensor.model.data.train.apply(kr_tensor, tensor)

# Define the learning function
model = kr_learn_model.classes.Kernel.UniformClassifier

kr_tensor = kr_tensor.model.data.train.apply(kr_tensor, kr_tensor_data)

# Define the traini

# 전체 코드

In [ ]:
# 데이터세트 로딩
from datasets import load_dataset

train_dataset = load_dataset('Aeala/ShareGPT_Vicuna_unfiltered', split="train[:10000]")
train_dataset


# 토크나이저 로딩
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
tokenizer


# 모델 로딩
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('gpt2')
model



# 데이터 전처리

"""
HUMAN: ~~~~  \n
CHATBOT: ~~~~ \n
"""

tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
	conversations = examples["conversations"]
	text = ''
	for item in conversations:
		if item['from'] == "human":
			text += f"HUMAN: {item['value']} \n"
		else:
			text += f"CHATBOT: {item['value']} \n"
	return tokenizer(text, padding="max_length", truncation=True, max_length=512)


tokenized_datasets = train_dataset.map(
		tokenize_function,
		remove_columns=["conversations", "id"]
)
tokenized_datasets


# 배치 처리
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
data_collator



# 모델 학습
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
	output_dir="./gpt2_finetuned",
	overwrite_output_dir=True,
	num_train_epochs=2,
	per_device_train_batch_size=8,
	save_steps=1000,
	save_total_limit=2,
)

# Trainer 객체를 생성
trainer = Trainer(
	model=model,
	args=training_args,
	data_collator=data_collator,
	train_dataset=tokenized_datasets,
)

# 학습 시작
trainer.train()